# Model 1 Using ANN and LTSM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import yfinance as yf
import seaborn as sns
%matplotlib inline

In [ ]:
start_training = datetime.date(2021, 10, 14)
end_training = datetime.date(2022, 7, 1)
start_testing = datetime.date(2022, 7, 2)
end_testing = datetime.datetime.today()
ticker = "NVDA"

In [ ]:
df_training = yf.download(ticker, start=start_training, end=end_training, progress=False)
print(f"Downloaded {df_training.shape[0]} rows and {df_training.shape[1]} columns of {ticker} data")


df_testing = yf.download(ticker, start=start_testing, end=end_testing, progress=False)
print(f"Downloaded {df_testing.shape[0]} rows and {df_testing.shape[1]} columns of {ticker} data")

In [ ]:
dataset = df_training[['Close','Volume']]
training_set = dataset.iloc[:, 0:1].values

testset = df_testing[['Close','Volume']]
testset = testset.iloc[:, 0:1].values

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
testset_scaled = sc.fit_transform(testset)

In [ ]:
training_set.size

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, int(training_set.size)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
X_test1 = []
y_test1 = []
for i in range(60, int(testset.size)):
    X_test1.append(testset_scaled[i-60:i, 0])
    y_test1.append(testset_scaled[i, 0])
X_test1, y_test1 = np.array(X_test1), np.array(y_test1)

In [ ]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

In [ ]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [ ]:

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [ ]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model = regressor.fit(x = X_train, y = y_train, epochs = 30, batch_size = 60, 
              validation_data=(X_test1, y_test1), verbose = 1,
              callbacks=[early_stop])


In [ ]:
model_loss = pd.DataFrame(model.history)
model_loss.plot()

In [ ]:
# Getting the predicted stock price of 2017
from imghdr import tests


dataset_total = pd.concat((df_training['Close'], df_testing['Close']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(df_testing) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 60+int(testset.size)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)


In [ ]:
X_f = []
for i in range(60, 60+12):
    X_f.append(inputs[i-60:i, 0])
X_f = np.array(X_f)
X_f = np.reshape(X_f, (X_f.shape[0], X_f.shape[1], 1))
predicted_stock_price1 = regressor.predict(X_f)
predicted_stock_price1 = sc.inverse_transform(predicted_stock_price1)
predicted_stock_price1

In [ ]:
# Visualising the results
plt.plot(testset, color = 'red', label = 'Real NVDA Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted NVDA Stock Price')
plt.title('NVDA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('NVDA Stock Price')
plt.legend()
plt.show()

## Train on full dataset 

In [ ]:
start = datetime.date(2022, 7, 14)
end = datetime.datetime.today()

In [ ]:
full_data = yf.download(ticker, start=start, end=end, progress=False)
print(f"Downloaded {full_data.shape[0]} rows and {full_data.shape[1]} columns of {ticker} data")

In [ ]:
dataset = full_data[['Close','Volume']]
dataset = dataset.iloc[:, 0:1].values

In [ ]:
full_data[['Close']].plot()

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
training_scaled = sc.fit_transform(dataset)

In [ ]:
X1 = []
y1 = []
for i in range(60, int(dataset.size)):
    X1.append(training_scaled[i-60:i, 0])
    y1.append(training_scaled[i, 0])
X1, y1 = np.array(X1), np.array(y1)

In [ ]:
X1 = np.reshape(X1, (X1.shape[0], X1.shape[1], 1))

In [ ]:
# Initialising the RNN
regressor1 = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor1.add(LSTM(units = 50, return_sequences = True, input_shape = (X1.shape[1], 1)))
regressor1.add(Dropout(0.2))

In [ ]:
# Adding a second LSTM layer and some Dropout regularisation
regressor1.add(LSTM(units = 50, return_sequences = True))
regressor1.add(Dropout(0.2))

In [ ]:
# Adding a third LSTM layer and some Dropout regularisation
regressor1.add(LSTM(units = 50, return_sequences = True))
regressor1.add(Dropout(0.2))

In [ ]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor1.add(LSTM(units = 50))
regressor1.add(Dropout(0.2))

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
# Adding the output layer
regressor1.add(Dense(units = 1))

# Compiling the RNN
regressor1.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
final = regressor1.fit(x=X1, y=y1, epochs = 30, batch_size = 60)

In [ ]:
inputs = full_data[len(full_data) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
result = []
for i in range(len(full_data)-10, len(full_data)):
    result.append(inputs[i-60:i, 0])
result = np.array(result)
result = np.reshape(result, (result.shape[0], result.shape[1], 1))
output = regressor1.predict(result)
output = sc.inverse_transform(output)

In [ ]:
prediction = pd.DataFrame(output)
prediction = prediction.rename(columns={0: 'Close'})

In [ ]:
prediction.reset_index()

In [ ]:
prediction.to_csv('model1.csv',index=False)